In [1]:
from huggingface_hub import snapshot_download
import boto3

from dataclasses import dataclass
from typing import Optional

import os
import time
from dotenv import load_dotenv

In [2]:
load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv('aws_access_key_id')
AWS_SECRET_ACCESS_KEY = os.getenv('aws_secret_access_key')
ENDPOINT_URL = os.getenv('endpoint_url')

In [3]:
@dataclass
class Config:
    #model_name: str = "facebook/opt-30b"
    model_name: str = "gpt2"
    # Path to HuggingFace cache directory. Default is ~/.cache/huggingface/hub.
    cache_dir: Optional[str] = '/data/modelcache/huggingface/hub' #None
    # Path to the directory that actually holds model files.
    # e.g., ~/.cache/huggingface/models--facebook--opt-30b/snapshots/xxx/
    # If this path is not None, we skip download models from HuggingFace.
    repo_root: Optional[str] = None
    # This is how many DeepSpeed-inference replicas to run for
    # this batch inference job.
    num_worker_groups: int = 1
    # Number of DeepSpeed workers per group.
    num_workers_per_group: int = 8

    batch_size: int = 1
    dtype: str = "float16"
    # Maximum number of tokens DeepSpeed inference-engine can work with,
    # including the input and output tokens.
    max_tokens: int = 1024
    # Use meta tensors to initialize model.
    use_meta_tensor: bool = True
    # Use cache for generation.
    use_cache: bool = True
    # The path for which we want to save the loaded model with a checkpoint.
    save_mp_checkpoint_path: Optional[str] = None


config = Config()

In [4]:
def download_model(config: Config):
    # This function downloads the specified HF model into a local directory.
    # This can also download models from cloud storages like S3.
    return snapshot_download(
        repo_id=config.model_name,
        cache_dir=config.cache_dir,
        allow_patterns=["*"],
        # Skip downloading TF and FLAX weight files.
        ignore_patterns=["*.safetensors", "*.msgpack", "*.h5"],
        revision=None,
    )

In [5]:
model_path = download_model(config)
model_path

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

'/data/modelcache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8'

In [6]:
session = boto3.Session(    
    aws_access_key_id = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
)

s3_client = session.client(
    service_name='s3',
    endpoint_url= ENDPOINT_URL,
)

s3_client.upload_file(Filename='test.txt', Bucket='models', Key='test.txt')

In [7]:
import os, botocore
def upload_files(path):
    for subdir, dirs, files in os.walk(path):
        for file in files:
            full_path = os.path.join(subdir, file)
            try:
                s3_client.head_object(Bucket='models', Key=full_path[len(path)+1:])
                #print(f'Existing: {full_path[len(path)+1:]}')
            except botocore.exceptions.ClientError as e:
                if e.response['Error']['Code'] == "404":
                    # The key does not exist.
                    print(f'Uploading: {full_path[len(path)+1:]}')
                    s3_client.upload_file(Filename=full_path, Bucket='models', Key=full_path[len(path)+1:])
                elif e.response['Error']['Code'] == "403":
                    # Unauthorized, including invalid bucket
                    print(e)
                else:
                  # Something else has gone wrong.
                    raise e    
                    

In [8]:
#upload_files('/home/arylwen/.cache/huggingface')
start = time.time()
upload_files('/data/modelcache/huggingface')
end = time.time()
print(f'elapsed: {end-start}')

Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/64-fp16.tflite
Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/64.tflite
Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/generation_config.json
Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/merges.txt
Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/pytorch_model.bin
Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/README.md
Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/rust_model.ot
Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/tokenizer.json
Uploading: hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/vocab.json
elapsed: 429.85366702079773
